In [164]:
from optimade.filter import Parser
from lark import Lark, Transformer

p = Parser(version=(0, 9, 5))
# (x<3  and  y!=2)  or  not  z<0
#
t1 = p.parse("filter=a<0 AND b>= 3")
print(p)

start
  filter=
  expression
    term
      term
        atom
          comparison
            a
            <
            0
      AND
      atom
        comparison
          b
          >=
          3



In [165]:
switcher = {
    "=": "$eq",
    ">":"$gt",
    ">=":"$gte",
    "<":"$lt",
    "<=":"$lte",
    "!=":"$ne",
    "AND":"$and",
    "and":"$and",
    "not":"$not",
    "NOT":"$not",
    "or":"$or",
    "OR":"$or",
}
def operatorToMongoDBSyntax(operator):
    return switcher[operator]

def listToString(l):
    return ''.join(l)

In [166]:
class EmptyTransformer(Transformer):
    def comparison(self, args):
        return args

In [186]:
class MyTransformer(Transformer):
    def comparison(self, args):
        A = args[0]
        operator = args[1]
        try:
            operator = operatorToMongoDBSyntax(args[1])
        except:
            print("Invalid operator detected: " + operator)
        B = args[2]
        return "{{{}: {{{}: {}}}}}".format(A, operator, B)
    def atom(self, args):
        return args[0]
    def term(self, args):
        if(len(args) == 1):
            # if the term is just an atom
            return args[0]
        elif(len(args) == 3):
            conjunction = operatorToMongoDBSyntax(args[1])
            A = args[0]
            B = args[2]
            print("{{ {}: [{},{}]}}".format(conjunction, A, B))
            return "{{ {}: [{},{}]}}".format(conjunction, A, B)
    def expression(self,args):
        return args[0]
    def start(self, args):
        return args[1]

In [187]:
transformer = MyTransformer().transform(p.tree)

{ $and: [{a: {$lt: 0}},{b: {$gte: 3}}]}


In [188]:
print(transformer)

{ $and: [{a: {$lt: 0}},{b: {$gte: 3}}]}


In [189]:
transformer

'{ $and: [{a: {$lt: 0}},{b: {$gte: 3}}]}'